# V3 Auto Category Table

In [1]:
# Import Libraries
########################################
#         Data Import from Sheets      #
########################################
import gspread
import pandas as pd
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.message import EmailMessage
import os
import mimetypes
from pathlib import Path
import json
from dotenv import load_dotenv

load_dotenv()

C:\Users\lohan\AppData\Local\Temp\ipykernel_12880\3687948299.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


True

In [2]:
# Credentials for Importing Google Sheets Data
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv('TRY'), scope)
json_keyfile_dict = json.loads(os.getenv("JSON_KEYFILE_DICT"))
creds = ServiceAccountCredentials.from_json_keyfile_dict(json_keyfile_dict, scope)
client = gspread.authorize(creds)
spreadsheet_key = os.getenv("SPREADSHEET_KEY2")
spreadsheet = client.open_by_key(spreadsheet_key)

In [3]:
# Import Expenses Data
sheet_title = 'Transactions'  
sheet = spreadsheet.worksheet(sheet_title)
expenses_data = sheet.get_all_records()
expenses_data = pd.DataFrame(expenses_data)

# Clean Time data
expenses_data["Date"] = expenses_data["Date"].astype(str).str.strip()

expenses_data["Date"] = pd.to_datetime(
    expenses_data["Date"],
    format="mixed",            
    utc=True,                  
    dayfirst=False          
)

expenses_data["Date"] = expenses_data["Date"].dt.tz_convert(None)

# Remove Money Hold if applicable
expenses_data = expenses_data[expenses_data['Category'] != "Hold"]

# expenses_data["Date"] = pd.to_datetime(expenses_data['Date'])

In [25]:
import pandas as pd
from datetime import datetime

df = expenses_data

# Ensure 'Date' is datetime
df['Date'] = pd.to_datetime(df['Date'])

# Current year and month
now = datetime.now()
current_year = now.year
current_month = now.month

# ——————————————————————————————
# 1) Summaries for year & total (you already have)

# This year
df_this_year = df[df['Date'].dt.year == current_year]
this_year_summary = (
    df_this_year
    .groupby('Category')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Spent This Year'})
)

# All-time
total_summary = (
    df
    .groupby('Category')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Spent Total'})
)

# Merge year & total
summary = total_summary.merge(
    this_year_summary,
    on='Category',
    how='left'
).fillna({'Spent This Year': 0})

# ——————————————————————————————
# 2) New: month-to-date

df_this_month = df[
    (df['Date'].dt.year == current_year) &
    (df['Date'].dt.month == current_month)
]
this_month_summary = (
    df_this_month
    .groupby('Category')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Spent This Month'})
)

# Merge month
summary = summary.merge(
    this_month_summary,
    on='Category',
    how='left'
).fillna({'Spent This Month': 0})

summary = summary.sort_values(by='Spent This Year', ascending=False)

# ——————————————————————————————
# 3) Percentages

# % of total (all-time)
total_all = summary['Spent Total'].sum()
summary['% Total'] = (summary['Spent Total'] / total_all * 100).round(2)

# % of year
total_year = summary['Spent This Year'].sum()
summary['% Year'] = (summary['Spent This Year'] / total_year * 100).fillna(0).round(2)

# % of month
total_month = summary['Spent This Month'].sum()
summary['% Month'] = (summary['Spent This Month'] / total_month * 100).fillna(0).round(2)

# ——————————————————————————————
# 4) Totals row

totals = pd.DataFrame({
    'Category': ['Total'],
    'Spent This Month': [summary['Spent This Month'].sum()],
    'Spent This Year' : [summary['Spent This Year'].sum()],
    'Spent Total'     : [summary['Spent Total'].sum()],
    '% Month'         : [100.0],
    '% Year'          : [100.0],
    '% Total'         : [100.0],
})

# Append and sort
summary = pd.concat([summary, totals], ignore_index=True)


# ——————————————————————————————
# 5) Final column order
column_order = [
    'Category',
    'Spent This Month', '% Month',
    'Spent This Year', '% Year',
    'Spent Total',      '% Total',
]
summary_total = summary[column_order]



In [26]:
df = expenses_data

df = df[df['Outlier'] != 1]

# Ensure 'Date' is datetime
df['Date'] = pd.to_datetime(df['Date'])

# Current year and month
now = datetime.now()
current_year = now.year
current_month = now.month

# ——————————————————————————————
# 1) Summaries for year & total (you already have)

# This year
df_this_year = df[df['Date'].dt.year == current_year]
this_year_summary = (
    df_this_year
    .groupby('Category')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Spent This Year'})
)

# All-time
total_summary = (
    df
    .groupby('Category')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Spent Total'})
)

# Merge year & total
summary = total_summary.merge(
    this_year_summary,
    on='Category',
    how='left'
).fillna({'Spent This Year': 0})

# ——————————————————————————————
# 2) New: month-to-date

df_this_month = df[
    (df['Date'].dt.year == current_year) &
    (df['Date'].dt.month == current_month)
]
this_month_summary = (
    df_this_month
    .groupby('Category')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Spent This Month'})
)

# Merge month
summary = summary.merge(
    this_month_summary,
    on='Category',
    how='left'
).fillna({'Spent This Month': 0})

summary = summary.sort_values(by='Spent This Year', ascending=False)

# ——————————————————————————————
# 3) Percentages

# % of total (all-time)
total_all = summary['Spent Total'].sum()
summary['% Total'] = (summary['Spent Total'] / total_all * 100).round(2)

# % of year
total_year = summary['Spent This Year'].sum()
summary['% Year'] = (summary['Spent This Year'] / total_year * 100).fillna(0).round(2)

# % of month
total_month = summary['Spent This Month'].sum()
summary['% Month'] = (summary['Spent This Month'] / total_month * 100).fillna(0).round(2)

# ——————————————————————————————
# 4) Totals row

totals = pd.DataFrame({
    'Category': ['Total'],
    'Spent This Month': [summary['Spent This Month'].sum()],
    'Spent This Year' : [summary['Spent This Year'].sum()],
    'Spent Total'     : [summary['Spent Total'].sum()],
    '% Month'         : [100.0],
    '% Year'          : [100.0],
    '% Total'         : [100.0],
})

# Append and sort
summary = pd.concat([summary, totals], ignore_index=True)


# ——————————————————————————————
# 5) Final column order
column_order = [
    'Category',
    'Spent This Month', '% Month',
    'Spent This Year', '% Year',
    'Spent Total',      '% Total',
]
summary_no_out = summary[column_order]


C:\Users\lohan\AppData\Local\Temp\ipykernel_12880\3125728656.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])


In [27]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


# 2. Convert to HTML with inline CSS for gridlines
html_table = summary_no_out.to_html(index=False, border=0)
html_table2 = summary_total.to_html(index=False, border=0)
styled_html = f"""
<html>
  <head>
    <style>
      table, th, td {{
        border: 1px solid black;
        border-collapse: collapse;
        padding: 4px;
      }}
      th {{ background-color: #f2f2f2; }}
    </style>
  </head>
  <body>
    <p>Hi there,</p>
    <p>No Outliers:</p>
    {html_table}
    <p>With Outliers:</p>
    {html_table2}
    <p>Best,<br>Your Python Script</p>
  </body>
</html>
"""

# 3. Build the email message
msg = MIMEMultipart('alternative')
msg['Subject'] = "Category Expenses Overall"
msg['From']    = "lorossiprojects@gmail.com"
msg['To']      = "lohan.rossi@hotmail.com"

# Attach the HTML part
msg.attach(MIMEText(styled_html, 'html'))

# 4. Send via SMTP (example: Gmail SMTP server)
smtp_server = "smtp.gmail.com"
smtp_port   = 587
smtp_user   = "lorossiprojects@gmail.com"
smtp_pass   = "esso ebao dfml ccwj"

with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.ehlo()
    server.starttls()
    server.login(smtp_user, smtp_pass)
    server.send_message(msg)

print("Email sent successfully!")


Email sent successfully!
